## Завдання
Проведіть класифікацію на основі датасету UCI ML Breast Cancer Wisconsin, використовуючи вказані у презентації моделі бустингу, беггінгу та стекінгу. Змініть їх параметри наступним чином:  
+ у StackingClassifier як базові алгоритми встановіть LogisticRegression та DecisionTreeClassifier, як фінальну модель – SVC;
+ у BaggingClassifier як базовий алгоритм встановіть DecisionTreeClassifier;
+ у AdaBoostClassifier як базовий алгоритм встановіть LogisticRegression
+ у XGBСlassifier як базовий алгоритм (booster) встановіть ‘gbtree’.
+ У зведеній таблиці порівняйте результати класифікацій, наведених у коді, із класифікаціями, що отримані вами.


In [118]:
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn import preprocessing

##### Загрузка та підготовка даних

In [119]:
data = load_breast_cancer()
X, y = load_breast_cancer(return_X_y=True)

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=12)

##### Stacking ensemble

In [120]:
estimators = [('lgr', LogisticRegression()), ('dtr', DecisionTreeClassifier())]
model_cls = StackingClassifier(estimators=estimators, final_estimator=SVC())

In [121]:
model_cls.fit(X_train, y_train)

StackingClassifier(estimators=[('lgr', LogisticRegression()),
                               ('dtr', DecisionTreeClassifier())],
                   final_estimator=SVC())

In [135]:
stc_score = model_cls.score(X_test, y_test)
print(stc_score)

0.9766081871345029


##### Bagging ensemble

In [123]:
model_bagcls = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=12)
model_bagcls.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=12)

In [136]:
bag_score = model_bagcls.score(X_test, y_test)
print(bag_score)

0.9005847953216374


##### AdaBoost ensemble

In [125]:
model_ada_bclf = AdaBoostClassifier(base_estimator=LogisticRegression(), n_estimators=10, random_state=12)
model_ada_bclf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=LogisticRegression(), n_estimators=10,
                   random_state=12)

In [137]:
adaBoost_score = model_ada_bclf.score(X_test, y_test)
print(adaBoost_score)

0.9766081871345029


##### XGBoost ensemble

In [127]:
model_xgdb_clf = XGBClassifier(booster='gbtree', learning_rate=1, n_estimators=10)

In [128]:
model_xgdb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [138]:
xgBoost_score = model_xgdb_clf.score(X_test, y_test)
print(xgBoost_score)

0.9122807017543859


In [169]:
ensembles = {
    'Ensemble':['StackingClassifier', 'BaggingClassifier', 'AdaBoostClassifier', 'XGBClassifier'],
    'Base_estimators': ['LogisticRegression, DecisionTreeClassifier', 'DecisionTreeClassifier', 'LogisticRegression', 'gbtree'],
    'Final_estimator': ['SVC', np.nan, np.nan, np.nan,],
    'Model_score': [stc_score, bag_score, adaBoost_score, xgBoost_score],
    'Notes': 'homework'}

hw_results = pd.DataFrame(ensembles)
les_results = pd.read_csv('les_results.csv')
res = pd.concat([les_results, hw_results])
res.sort_values('Ensemble').reset_index(drop=True)


,Ensemble,Base_estimators,Final_estimator,Model_score,Notes
0,AdaBoostClassifier,DecisionTreeClassifier,NaN,0.918129,lesson
1,AdaBoostClassifier,LogisticRegression,NaN,0.976608,homework
2,BaggingClassifier,LogisticRegression,NaN,0.982456,lesson
3,BaggingClassifier,DecisionTreeClassifier,NaN,0.900585,homework
4,StackingClassifier,"LogisticRegression, SGDClassifier",DecisionTreeClassifier,0.959064,lesson
5,StackingClassifier,"LogisticRegression, DecisionTreeClassifier",SVC,0.976608,homework
6,XGBClassifier,gblinear,NaN,0.959064,lesson
7,XGBClassifier,gbtree,NaN,0.912281,homework
